In [1]:
from abmax.structs import *
from abmax.functions import *
import jax.numpy as jnp
import jax.random as random
import jax
from flax import struct

import numpy as np
from collections import deque

In [2]:
# Creating Car Agent and objects Cell, Map

@struct.dataclass
class Car(Agent):
    """
    state :
    current location (Y, X),
    direction_of_heading: 0: up, 1: down, 2: left, 3: right
    chaos: 0: no chaos, 1: chaos
    speed: 0: stopped, 1: moving
    
    parameters:
    map: map of the environment
    car_length: length of the car
    destination: destination of the car (Y, X)
    """
    @staticmethod
    def create_agent(type, params, id, active_state, key, policy):
        key, subkey = random.split(key)
    
        def create_active_agent():
            subkey, *create_keys = random.split(subkey, 3) #3 will change based on need
            map = params['map']
            start_locations = params['start_locations']
            destinations = params['destinations']
            start = start_locations[jax.random.randint(create_keys[0], 1, 1, len(start_locations))[0]]
            destination = destinations[jax.random.randint(create_keys[1], 1, 1, len(destinations))[0]]
            state_content = {'current_location': start, 'direction_of_heading': 0, 'chaos': 0, 'speed': 0}


            
        
        def create_inactive_agent():
            pass
        
        def get_surroundings():
            pass
            """try :
                x, y = state_content['current_location']
                return [map[x, y+1], map[x+1, y+1], map[x+1, y], map[x, y-1], map[x-1, y-1], map[x-1, y], map[x-1, y+1], map[x+1, y-1]]
            return"""
            
 
        def create_inactive_agent():
            pass



    """
    Represents a car in the environment.
    """
    """
    coords: jnp.array # [Y, X] coordinates of cell in Map
    direction: int # -1: none, 0: down, 1: left, 2: up, 3: right
    chaos: int # not implemented
    speed: int # number of squares a car travels in a single tick 
    destination: jnp.array # [Y, X] coordinates of destination
    uturn: bool # whether or not a NonJax_Car is allowed on connection_road
    """

directions = jnp.array((jnp.array((1, 0)), jnp.array((0, -1)), jnp.array((-1, 0)), jnp.array((0, 1)))) # Down, Left, Up, Right

class Cell(struct.PyTreeNode):
    """
    Represents a single cell on the map. ((coord_y, coord_x), type, heading_direction, lane, car)
    """
    coords: jnp.array  # [Y, X] coordinates of cell in Map
    cell_type: int  # -1: inaccessible, 0: road, 1: entry_cell, 2: exit_cell
    heading_direction: int  # -1: none, 0: down, 1: left, 2: up, 3: right
    lane: int # Used to declare relative switching/turning directions, -1: solo_lane (= no lane switching), 0: middle_lane (= switching left and right), 1: left_border_lane (= switching right), 2: right_border_lane (= switching left)
    car: Car # Car object if occupied, otherwise None

    def __str__(self): 
        if self.car is not None: 
            return "X" 
        else:
            """
            if self.cell_type == -1: # Prints lane if not inaccessible
                return str(self.cell_type)
            else: 
                return str(self.lane)
            """
            # return str(self.heading_direction)  # Prints heading_direction
            
            if self.cell_type == 0:   # Prints X if car, lane if normal road type, cell_type for any other road types
                return str(self.lane)
            elif self.cell_type == 1:
                return "S"
            elif self.cell_type == 2:
                return "D"
            else: 
                return str(self.cell_type)
            

    def change_cell_direction_and_lane(self, heading_direction, lane):
        return Cell(self.coords, self.cell_type, heading_direction, lane, self.car)
    
    def same_coord_and_type(self, cell_b): # cell_coords and cell_type are the only static values
        Y, X = self.coords
        bY, bX = cell_b.coords
        return (Y == bY and X == bX and self.cell_type == cell_b.cell_type)

class Map(struct.PyTreeNode):
    """
    Represents the map as a grid of Cells.
    """
    grid: jnp.ndarray  # 2D array of Cells

    def __str__(self):
        """string_reprs = [[cell.printme() for cell in row] for row in self.grid]
        max_width = max(len(cell_str) for row in string_reprs for cell_str in row)
        return "\n".join([
            " ".join(f"{cell_str:>{max_width}}" for cell_str in row)
            for row in string_reprs
        ])"""
        
        string_reprs = [[str(cell) for cell in row] for row in self.grid]
        max_width = max(len(cell_str) for row in string_reprs for cell_str in row)
        return "\n".join([
            " ".join(f"{cell_str:>{max_width}}" for cell_str in row)
            for row in string_reprs
        ])

    def get_cell(self, coords):
        """Returns the Cell at (Y, X), or None if out of bounds."""
        Y, X = coords
        if 0 <= Y < len(self.grid) and 0 <= X < len(self.grid):
            return self.grid[Y][X]
        return None
    
    def occupy(self, cell, car):
        """Assigns a car to this cell, returning a new occupied cell."""
        if cell.car is None:
            grid_copy = self.grid.copy()
            Y, X = cell.coords
            grid_copy[Y][X] = Cell(cell.coords, cell.cell_type, cell.heading_direction, cell.lane, car)
            return Map(grid_copy)
        else:
            return self

    def unoccupy(self, cell):
        """Removes the car from this cell, returning a new unoccupied cell."""
        if cell.car is not None:
            grid_copy = self.grid.copy()
            Y, X = cell.coords
            grid_copy[Y][X] = Cell(cell.coords, cell.cell_type, cell.heading_direction, cell.lane, None)
            return Map(grid_copy)
        else:
            return self
    
    def red_light_intersection(self, intersection_base, intersection_length):
        """Sets red_light to all Cells in intersection."""
        grid_copy = self.grid.copy()
        for i in range(intersection_length):
            for j in range(intersection_length):
                grid_copy[intersection_base[0] + i, intersection_base[1] + j] = self.grid[intersection_base[0] + i, intersection_base[1] + j].change_cell_direction_and_lane(heading_direction=-1, lane=-1)
        return Map(grid_copy)
    
    def no_light_intersection(self, intersection_base, intersection_length):
        grid_copy = self.grid.copy()
        lanes = intersection_length / 2
        for i in range(intersection_length):
            for j in range(intersection_length):
                # Dividing into quadrants top_left = 0, top_right = 1, bottom_right = 2, bottom_left = 3 (synonomous with heading_direction: down, left, up, right)
                check_Y = jnp.floor(i / lanes) 
                check_X = jnp.floor(j / lanes)
                lane = 0 # Every value is 0, except the leftmost column for each relative position
                if check_Y and check_X:
                    quad = 2 # Bottom right quadrant, goes up                               0 0 1   0 0 0   | Quadrant's zero rows/columns, examplified with length=6    1 < 1  | Minimal length = 2
                    if j == lanes: # relative left for upward direction is global left      0 0 1 < 0 0 0   | Top_left     (0) = quadrant's right column (lanes-1)       v   ^  |
                        lane = 1 #                                                          0 0 1   1 1 1   | Top_right    (1) = quadrant's bottom row (lanes-1)         1 > 1  |
                elif check_Y: #                                                               v       ^
                    quad = 3 # Bottom left quadrant, goes right                             1 1 1   1 0 0   | Bottom_left  (3) = quadrant's top row (lanes)
                    if i == lanes: # relative left for rightward direction is global up     0 0 0 > 1 0 0   | Bottom_right (2) = quadrant's left column (lanes)
                        lane = 1 #                                                          0 0 0   1 0 0   | Proud of this design <3
                elif check_X:
                    quad = 1 # Top right quadrant, goes left
                    if i == (lanes-1): # relative left for leftward direction is global down
                        lane = 1
                else:
                    quad = 0 # Top left quadrant, goes down
                    if j == (lanes-1): # relative left for downward direction is global right
                        lane = 1
                
                grid_copy[intersection_base[0] + i, intersection_base[1] + j] = self.grid[intersection_base[0] + i, intersection_base[1] + j].change_cell_direction_and_lane(heading_direction=quad, lane=lane)
        return Map(grid_copy)
        
    def find_successors(self, coords):
        Y, X = coords
        cell = self.grid[Y][X]
        cell_type = cell.cell_type # -1: inaccessible, 0: road, 1: entry_cell, 2: exit_cell, 3: intersection_cell
        hd = cell.heading_direction  # -1: none, 0: down, 1: left, 2: up, 3: right
        lane = cell.lane  # Used to declare relative switching/turning directions, -1: solo_lane (= no lane switching), 0: middle_lane (= switching left and right), 1: left_border_lane (= switching right), 2: right_border_lane (= switching left)
        
        if cell_type == -1 or hd == -1 or cell_type == 2:
            return cell
        else:
            successors = []
            direction = directions[hd]
            forward_coords = jnp.add(coords, direction) 

            forward_cell = self.get_cell(forward_coords)
            if forward_cell is not None:
                successors.append(forward_cell)

            if lane == 1 or lane == 0: # Rightward lane switch
                relative_right = directions[(hd+1)%4]
                right_switch_coords = jnp.add(forward_coords, relative_right)
                right_switch_cell = self.get_cell(right_switch_coords)
                if right_switch_cell is not None:
                    successors.append(right_switch_cell)

                if cell_type == 3: # Switching relative right in intersection doesn't take you in heading direction, as it is a valid turn
                    right_switch_coords = jnp.add(coords, relative_right)
                    right_switch_cell = self.get_cell(right_switch_coords)
                    if right_switch_cell is not None:
                        successors.append(right_switch_cell)

            if lane == 2 or lane == 0: # Leftward lane switch
                relative_left = directions[(hd-1)%4]
                left_switch_coords = jnp.add(forward_coords, relative_left)

                left_switch_cell = self.get_cell(left_switch_coords)
                if left_switch_cell is not None:
                    successors.append(left_switch_cell)

            if cell.car is not None: # U-turn
                if cell.car.uturn:
                    if cell_type != 3 and lane == 1:
                        relative_left = directions[(hd-1)%4]
                        left_switch_coords = jnp.add(forward_coords, relative_left)

                        left_switch_cell = self.get_cell(left_switch_coords)
                        if left_switch_cell is not None:
                            successors.append(left_switch_cell)
            return successors
        
    def shortest_path(self, start, end):
        if self.get_cell(start.coords) is None or self.get_cell(end.coords) is None:
            return None
        else:
            queue = deque([(start, [])])
            visited = jnp.zeros(2, 1)
            visited[0, :] = [start.coords]

            while queue:
                current, path = queue.popleft()
                path = path + [current.coords]  

                if current.same_coord_and_type(end):
                    return path
                
                for neighbor in self.find_successors(current.coords):
                    if neighbor.coords.all not in visited:
                        visited[-1, :] = [neighbor.coords]
                        visited = jnp.array[visited]
                        queue.append((neighbor, path))
            return None



In [3]:
def square_map(nr_junctions, nr_lanes, connection_length):
    # (self, car, coords, direction, relative_coords, quadrant)
    # Initializing map as a square matrix of size based on parameters with standard road value (2)
    # Cell((coord_y, coord_x), type, heading_direction, lane, car)
    two_lanes = 2 * nr_lanes
    map_length = nr_junctions * two_lanes + (nr_junctions + 1) * connection_length
    cell_list = np.empty((map_length, map_length), dtype=Cell)

    final_connection = nr_junctions*(connection_length + two_lanes)
    intersection_bases = []

    for base_Y in range(0, map_length, (connection_length + two_lanes)):
        if (base_Y == final_connection):
            lowest_connection = True
        else:
            lowest_connection = False

        for base_X in range(0, map_length, (connection_length + two_lanes)):
            if (base_X == final_connection):
                rightmost_connection = True
            else:
                rightmost_connection = False


            # Setting connection Cells
            if not lowest_connection: # Adding horizontal connection
                for lane_offset in range(two_lanes):
                    for connection_offset in range(connection_length):
                        X = base_X + connection_offset

                        # Setting lane
                        if nr_lanes > 1:
                            if lane_offset == 0 or lane_offset == two_lanes-1:
                                lane = 2 # Right border lane
                            elif lane_offset == nr_lanes-1 or lane_offset == nr_lanes:
                                    lane = 1 # Left border lane
                            else:
                                lane = 0 # Middle lane
                        else:
                            lane = -1 # Solo lane
                        
                        # Seperating the two driving direction connections
                        if lane_offset < nr_lanes: # Top half connection
                            direction = 1 # Global heading_direction is left

                            # Setting cell_type
                            if X == 0: # If in first column, it is an exit cell (2)
                                cell_type = 2 
                            elif X == map_length-1: # If in last column, it is an entry cell (1)
                                cell_type = 1 
                            else: # In any other case, it is a regular road cell (0)
                                cell_type = 0

                        else: # Bottom half connection
                            direction = 3 # Global heading_direction is right
    
                            # Setting cell_type 
                            if X == 0: # If in first column, it is an entry cell (1)
                                cell_type = 1
                            elif X == map_length-1: # If in last column, it is an exit cell (2)
                                cell_type = 2
                            else: # In any other case, it is a regular road cell (0)
                                cell_type = 0

                        cell_list[base_Y + connection_length + lane_offset, X] = Cell(coords=jnp.array((base_Y + connection_length + lane_offset, X)), cell_type=cell_type, heading_direction=direction, lane=lane, car=None)
            
            if not rightmost_connection: # Adding vertical connection
                for connection_offset in range(connection_length):
                    for lane_offset in range(two_lanes):
                        Y = base_Y + connection_offset
                        # Setting lane
                        if nr_lanes > 1:
                            if lane_offset == 0 or lane_offset == two_lanes-1:
                                lane = 2 # Right border lane
                            elif lane_offset == nr_lanes-1 or lane_offset == nr_lanes:
                                    lane = 1 # Left border lane
                            else:
                                lane = 0 # Middle lane
                        else:
                            lane = -1 # Solo lane
                        
                        # Seperating the two driving direction connections
                        if lane_offset < nr_lanes: # Left half connection
                            direction = 0 # Global heading_direction is down

                            # Setting cell_type
                            if Y == 0: # If in first row, it is an entry cell (1)
                                cell_type = 1
                            elif Y == map_length-1: # If in last row, it is an exit cell (2)
                                cell_type = 2
                            else: # In any other case, it is a regular road cell (0)
                                cell_type = 0

                        else: # Bottom half connection
                            direction = 2 # Global heading_direction is up
    
                            # Setting cell_type 
                            if Y == 0: # If in first row, it is an exit cell (2)
                                cell_type = 2 
                            elif Y == map_length-1: # If in last row, it is an entry cell (1)
                                cell_type = 1 
                            else: # In any other case, it is a regular road cell (0)
                                cell_type = 0
                        
                        cell_list[Y, base_X + connection_length + lane_offset] = Cell(jnp.array((Y, base_X + connection_length + lane_offset)), cell_type=cell_type, heading_direction=direction, lane=lane, car=None)

            # Setting inaccessible Cells
            for i in range(connection_length):
                for j in range(connection_length):
                    cell_list[base_Y + i, base_X + j] = Cell(jnp.array((base_Y + i, base_X + j)), -1, -1, -1, None)

            # Setting intersection Cells, initialized with heading_direction and lane being -1, to be decorated later using Map.no_light_intersection(intersection_base, intersection_length)
            if not rightmost_connection and not lowest_connection:
                intersection_base_Y = base_Y + connection_length
                intersection_base_X = base_X + connection_length
                intersection_bases.append((intersection_base_Y, intersection_base_X))
                for i in range (two_lanes):
                    for j in range (two_lanes):
                        cell_list[intersection_base_Y + i, intersection_base_X + j] = Cell(jnp.array((intersection_base_Y + i, intersection_base_X + j)), 3, -1, -1, None)
    map = Map(cell_list)
    for intersection_base in intersection_bases:
        map = map.no_light_intersection(intersection_base=intersection_base, intersection_length=two_lanes)
    return map

In [4]:
# Test zone
tester = square_map(nr_junctions=2, nr_lanes=3, connection_length=3)
print(tester)

print("\n","\n","\n")
minimal = square_map(nr_junctions=1, nr_lanes=1, connection_length=1)
print(minimal)

print("\n","\n","\n")
#tester = tester.red_light_intersection((3,3), 6)
#print(tester)

-1 -1 -1  S  S  S  D  D  D -1 -1 -1  S  S  S  D  D  D -1 -1 -1
-1 -1 -1  2  0  1  1  0  2 -1 -1 -1  2  0  1  1  0  2 -1 -1 -1
-1 -1 -1  2  0  1  1  0  2 -1 -1 -1  2  0  1  1  0  2 -1 -1 -1
 D  2  2  3  3  3  3  3  3  2  2  2  3  3  3  3  3  3  2  2  S
 D  0  0  3  3  3  3  3  3  0  0  0  3  3  3  3  3  3  0  0  S
 D  1  1  3  3  3  3  3  3  1  1  1  3  3  3  3  3  3  1  1  S
 S  1  1  3  3  3  3  3  3  1  1  1  3  3  3  3  3  3  1  1  D
 S  0  0  3  3  3  3  3  3  0  0  0  3  3  3  3  3  3  0  0  D
 S  2  2  3  3  3  3  3  3  2  2  2  3  3  3  3  3  3  2  2  D
-1 -1 -1  2  0  1  1  0  2 -1 -1 -1  2  0  1  1  0  2 -1 -1 -1
-1 -1 -1  2  0  1  1  0  2 -1 -1 -1  2  0  1  1  0  2 -1 -1 -1
-1 -1 -1  2  0  1  1  0  2 -1 -1 -1  2  0  1  1  0  2 -1 -1 -1
 D  2  2  3  3  3  3  3  3  2  2  2  3  3  3  3  3  3  2  2  S
 D  0  0  3  3  3  3  3  3  0  0  0  3  3  3  3  3  3  0  0  S
 D  1  1  3  3  3  3  3  3  1  1  1  3  3  3  3  3  3  1  1  S
 S  1  1  3  3  3  3  3  3  1  1  1  3  3  3  3  3  3  

In [5]:
my_point = jnp.array((4, 8))
my_cell = tester.get_cell(my_point)

accessible_neighbours = tester.find_successors(my_point)
for neigh in accessible_neighbours:
    print(neigh.coords)

[4 7]
[3 7]
[3 8]
[5 7]


In [6]:
my_start_point = tester.get_cell(jnp.array((6, 0)))
my_end_point = tester.get_cell(jnp.array((13, 0)))

sandbox = jnp.zeros((2, 2))


#pathie =  tester.shortest_path(my_start_point, my_end_point)

"""
for cello in pathie:
    print(cello.coords, end="\t")"
"""

'\nfor cello in pathie:\n    print(cello.coords, end="\t")"\n'

In [7]:
# Outdated code
"""
# Creating functions that support or interpret objects and agent's actions 


def logic(matrix, Y, X, prev_Y, prev_X, destination):
    if not (0 <= Y <= len(matrix.grid) and 0 <= X <= len(matrix.grid)):  # Out of bounds
        return False
    # Prevent switching between 1 and 2 directly
    
    prev_cell = matrix.get_cell(prev_Y, prev_X)
    curr_cell = matrix.get_cell(Y, X)
    prev_value = prev_cell.map_info
    curr_value = curr_cell.map_info
    planned_direction = DIRECTIONS.index((Y-prev_Y, X-prev_X))

    if (curr_cell.get_coords() == destination):
        return True
    
    # Rejecting moves based on road type and previous behavior
    if curr_value == -1:
        return False # Inaccessible
    if (curr_value == 3 and prev_value != 3):
        return False # No need to visit start tiles unless you are navigating away from start
    if (curr_value == 4 and prev_value == 3) or (curr_value == 2 and prev_value == 1) or (curr_value == 1 and prev_value == 2):
        return False # Passing from 3 to 4 / 1 to 2 and vice versa directly would require crossing lanes with different polarity
    if (curr_value == 4 and (Y, X) != destination):
        return False # Cannot enter terminal unless it's the destination
    if (curr_value == 1 and planned_direction in {1, 2}) or (curr_value == 2 and planned_direction in {3,0}): # If fixed its 3,4/1,2 - otherwise 12 30
        return False # 1 roads are North/East, 2 roads are South/West
    if curr_value == 0 and prev_value in {1, 2, 3}:
        right = DIRECTIONS[(planned_direction+1)%4]
        right_square_value = matrix.get_cell(prev_Y+right[0], prev_X+right[1]).map_info
        if right_square_value in {1, 2, 3} and right_square_value == prev_value:
            return False # Must be in rightmost lane when entering intersection
    return True

def find_route(matrix, start_3, target_4):
    queue = deque([(start_3, [start_3])])  # (current_position, path_so_far)
    visited = set()
    visited.add(start_3)

    while queue:
        (Y, X), path = queue.popleft()

        if (Y, X) == target_4:
            return path

        for dx, dy in DIRECTIONS:
            nX, nY = Y + dx, X + dy

            if logic(matrix, nX, nY, Y, X, target_4) and (nX, nY) not in visited:
                queue.append(((nX, nY), path + [(nX, nY)]))
                visited.add((nX, nY))

    return None  # No path found



test_map = create_square_problem(nr_junctions=2, nr_lanes=3, connecting_length=3)
cell_list = cell_list_from_square_array(test_map)
mapp = Map(cell_list)

start_locations = mapp.find_cell_info(3)
destinations = mapp.find_cell_info(4)

key = random.PRNGKey(2)
key, subkey = random.split(key)
start_point = start_locations[jax.random.randint(subkey, 1, 1, len(start_locations))[0]]
key, subkey = random.split(key)
destination = destinations[jax.random.randint(subkey, 1, 1, len(start_locations))[0]]

print(start_point, mapp.get_cell(start_point[0], start_point[1]).map_info, destination, mapp.get_cell(destination[0], destination[1]).map_info)

car = NonJAX_Car(start_point[0], start_point[1], direction=2, chaos=0, speed=1, destination=destination)
mapp = mapp.update_cell(start_point[0], start_point[1], mapp.get_cell(start_point[0], start_point[1]).occupy(car))
print(mapp)

print(mapp.get_cell(start_point[0], start_point[1]))
path = find_route(mapp, start_point, destination)
print("Path:", path)



visual = test_map.copy()
for coord in path:
    visual[coord[0], coord[1]] = 6
visual[start_point] = 7
visual[destination] = 8
print(visual)
"""

'\n# Creating functions that support or interpret objects and agent\'s actions \n\n\ndef logic(matrix, Y, X, prev_Y, prev_X, destination):\n    if not (0 <= Y <= len(matrix.grid) and 0 <= X <= len(matrix.grid)):  # Out of bounds\n        return False\n    # Prevent switching between 1 and 2 directly\n    \n    prev_cell = matrix.get_cell(prev_Y, prev_X)\n    curr_cell = matrix.get_cell(Y, X)\n    prev_value = prev_cell.map_info\n    curr_value = curr_cell.map_info\n    planned_direction = DIRECTIONS.index((Y-prev_Y, X-prev_X))\n\n    if (curr_cell.get_coords() == destination):\n        return True\n    \n    # Rejecting moves based on road type and previous behavior\n    if curr_value == -1:\n        return False # Inaccessible\n    if (curr_value == 3 and prev_value != 3):\n        return False # No need to visit start tiles unless you are navigating away from start\n    if (curr_value == 4 and prev_value == 3) or (curr_value == 2 and prev_value == 1) or (curr_value == 1 and prev_valu